<a href="https://colab.research.google.com/github/NatashaKamami/Music-Recommendation-System/blob/main/youtubescraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install yt-dlp youtube-search-python librosa


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.5/99.5 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.1/89.1 kB 5.7 MB/s eta 0:00:00


In [3]:
!pip install --force-reinstall httpx==0.23.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.4/68.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: sniffio
    Found existing installation: sniffio 1.3.1
    Uninstalling sniffio-1.3.1:
      Successfully uninstalled sniffio-1.3.1
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
  Attempting uninstall: h11
    Found existing installation: h11 0.14.0
    Uninstalling h11-0.14.0:
      Successfully uninstalled h11-0.14.0
  Attempti

In [1]:
import pandas as pd
import yt_dlp
import librosa
import numpy as np
from youtubesearchpython import VideosSearch
import os
import time
import librosa.feature



In [ ]:
# Load the CSV file
df = pd.read_csv("/content/pop_song_details.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 438 entries, 0 to 437
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   song_id     438 non-null    object
 1   name        438 non-null    object
 2   artist      438 non-null    object
 3   album       438 non-null    object
 4   popularity  438 non-null    int64 
dtypes: int64(1), object(4)
memory usage: 17.2+ KB


In [ ]:
# Folder to store downloaded audio
os.makedirs("downloads", exist_ok=True)

In [ ]:
# Function to search YouTube for a song
def search_youtube(song_name, artist):
    query = f"{song_name} {artist} official audio"
    search = VideosSearch(query, limit=1)
    result = search.result()

    if result["result"]:
        return result["result"][0]["link"]  # Return the top result URL
    return None  # No results found

In [ ]:
# Function to download audio from YouTube
def download_audio(youtube_url, output_path):
    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
            'nopostoverwrites': False  # Prevents adding extra extensions
        }],
        'outtmpl': "downloads/%(title)s.%(ext)s",  # Ensures correct naming
        'quiet': True  # Suppress output
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([youtube_url])


In [ ]:
# Function to extract audio features
def extract_audio_features(audio_path):
    try:
        # Extract Tempo (Beats Per Minute - BPM)
        onset_env = librosa.onset.onset_strength(y=y, sr=sr)
        tempo_array, _ = librosa.beat.beat_track(y=y, sr=sr, onset_envelope=onset_env)
        tempo = tempo_array.item()  # Fix: Extract single float value

        # Spectral Rolloff
        spectral_rolloff = np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr)).item()

        # Chroma Features
        chroma = np.mean(librosa.feature.chroma_stft(y=y, sr=sr)).item()

        # Energy
        energy = np.mean(librosa.feature.rms(y=y)).item()

        # Danceability Approximation
        max_onset = np.max(onset_env).item()  # Convert NumPy array to scalar
        danceability = tempo / (max_onset + 1) # calculate danceability

        return tempo, energy, spectral_rolloff, chroma, danceability
    except Exception as e:
        print(f"Error processing {audio_path}: {e}")
        return None, None, None, None, None # Return None values if extraction fails

In [ ]:
# Path to downloads folder
downloads_folder = "downloads"

# Function to find the correct file in the downloads folder
def find_audio_file(song_name, artist):
    all_files = os.listdir(downloads_folder)
    for file in all_files:
        lower_file = file.lower()
        if song_name.lower() in lower_file and artist.lower() in lower_file and file.endswith((".mp3", ".wav", ".flac", ".m4a")):
            return os.path.join(downloads_folder, file)
    return None  # Return None if no matching file is found

# Loop through the CSV and process each song
for index, row in df.iterrows():
    song_name = row["name"].strip().lower()
    artist = row["artist"].strip().lower()

    print(f"Processing: {song_name} - {artist}")

    # Step 1: Search for the file dynamically
    audio_path = find_audio_file(song_name, artist)

    if not audio_path:  # If file not found, download it
        youtube_url = search_youtube(song_name, artist)
        if not youtube_url:
            print(f"No YouTube link found for {song_name} - {artist}, skipping...")
            continue  # Skip if no link is found

        # Step 2: Download the audio
        try:
            print(f"Downloading: {youtube_url}")
            download_audio(youtube_url, downloads_folder)  # Save in downloads folder
            time.sleep(2)  # Wait for download to complete
        except Exception as e:
            print(f"Failed to download {song_name}: {e}")
            continue  # Skip if download fails

        # Search again after downloading
        audio_path = find_audio_file(song_name, artist)
        if not audio_path:
            print(f"Download completed but file not found in downloads folder for {song_name} - {artist}")
            continue

    print(f"Found audio file: {audio_path}")

    # Step 3: Extract audio features
    try:
        y, sr = librosa.load(audio_path)
        print(f"Audio loaded successfully! Length: {len(y)} samples, Sample rate: {sr}")

        # Extract features
        tempo, energy, spectral_rolloff, chroma, loudness, danceability = extract_audio_features(audio_path)
        print(f"Extracted features for {audio_path}")

        # Store in CSV
        df.loc[index, "youtube_url"] = youtube_url if "youtube_url" in df.columns else None
        df.loc[index, "tempo"] = tempo
        df.loc[index, "energy"] = energy
        df.loc[index, "spectral_rolloff"] = spectral_rolloff
        df.loc[index, "chroma"] = chroma
        df.loc[index, "danceability"] = danceability
    except Exception as e:
        print(f"Error processing {audio_path}: {e}")
        continue  # Skip if feature extraction fails

print("Processing complete!")


Processing: annihilate (spider-man: across the spider-verse) (metro boomin & swae lee, lil wayne, offset) - metro boomin
Downloading: https://www.youtube.com/watch?v=dsnuu20RSFU
Download completed but file not found in downloads folder for annihilate (spider-man: across the spider-verse) (metro boomin & swae lee, lil wayne, offset) - metro boomin
Processing: otaku hot girl - megan thee stallion
Downloading: https://www.youtube.com/watch?v=USxIYOaIpig
Found audio file: downloads/Megan Thee Stallion - Otaku Hot Girl [Official Audio].mp3
Audio loaded successfully! Length: 3498753 samples, Sample rate: 22050
Extracted features for downloads/Megan Thee Stallion - Otaku Hot Girl [Official Audio].mp3
Processing: squeeze (feat. megan thee stallion) - latto
Downloading: https://www.youtube.com/watch?v=2d7FCimHwO8
Download completed but file not found in downloads folder for squeeze (feat. megan thee stallion) - latto
Processing: love scars - trippie redd
Downloading: https://www.youtube.com/wat

ERROR: unable to download video data: HTTP Error 403: Forbidden


Failed to download passenger princess (jibbitz): ERROR: unable to download video data: HTTP Error 403: Forbidden
Processing: enough (miami) - cardi b
Downloading: https://www.youtube.com/watch?v=MLG_DsCQMsc
Found audio file: downloads/Cardi B - Enough (Miami) [Official Audio].mp3
Audio loaded successfully! Length: 3500033 samples, Sample rate: 22050
Extracted features for downloads/Cardi B - Enough (Miami) [Official Audio].mp3
Processing: god's plan - drake
Downloading: https://www.youtube.com/watch?v=m1a_GqJf02M
Download completed but file not found in downloads folder for god's plan - drake
Processing: tap in - saweetie
Downloading: https://www.youtube.com/watch?v=UGNPtonsylQ
Found audio file: downloads/Saweetie - Tap In [Official Lyrics Video].mp3
Audio loaded successfully! Length: 3197309 samples, Sample rate: 22050
Extracted features for downloads/Saweetie - Tap In [Official Lyrics Video].mp3
Processing: timeless (feat. playboi carti) - the weeknd
Downloading: https://www.youtube.

ERROR: unable to download video data: HTTP Error 403: Forbidden


Failed to download out my way: ERROR: unable to download video data: HTTP Error 403: Forbidden
Processing: murder on my mind - ynw melly
Downloading: https://www.youtube.com/watch?v=zFW5-ABDC-8
Found audio file: downloads/YNW Melly - Murder On My MInd (Audio).mp3
Audio loaded successfully! Length: 5908224 samples, Sample rate: 22050
Extracted features for downloads/YNW Melly - Murder On My MInd (Audio).mp3
Processing: would you? - lay bankz
Downloading: https://www.youtube.com/watch?v=e6EN1GtWmbs
Download completed but file not found in downloads folder for would you? - lay bankz
Processing: fair trade (with travis scott) - drake
Downloading: https://www.youtube.com/watch?v=THVbtGqEO1o
Download completed but file not found in downloads folder for fair trade (with travis scott) - drake
Processing: heaven or hell - k-trap
Downloading: https://www.youtube.com/watch?v=BCJtcPv7yHI
Found audio file: downloads/K-Trap - Heaven or Hell (Lyric Video).mp3
Audio loaded successfully! Length: 333607

ERROR: [youtube] ZR_X6PrASho: Sign in to confirm your age. This video may be inappropriate for some users. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies


Failed to download i don't fuck with you: ERROR: [youtube] ZR_X6PrASho: Sign in to confirm your age. This video may be inappropriate for some users. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies
Processing: jeffrey - juice wrld
Downloading: https://www.youtube.com/watch?v=LSXX4H58VB0
Found audio file: downloads/Juice WRLD - Jeffrey (Official Audio).mp3
Audio loaded successfully! Length: 3859969 samples, Sample rate: 22050
Extracted features for downloads/Juice WRLD - Jeffrey (Official Audio).mp3
Processing: syphilis - juice wrld
Downloading: https://www.youtube.com/watch?v=5qdavEWGyyI
Found audio file: downloads/Juice WRLD - Syphilis (Official Audio).mp3
Audio loaded successfully! Length: 2905661 samples, Sample rate: 2205

In [ ]:
df = df.dropna()
df = df.drop_duplicates()

In [ ]:
# Save the updated CSV with audio features
df.to_csv("pop_songs_with_features2.csv", index=False)
print("Updated CSV saved as 'pop_songs_with_features2.csv'")

Updated CSV saved as 'pop_songs_with_features2.csv'


In [2]:
# Loading the merged csv with all the songs
data = pd.read_csv("/content/songs_with_features.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5081 entries, 0 to 5080
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   song_id           5081 non-null   object 
 1   name              5081 non-null   object 
 2   artist            5081 non-null   object 
 3   album             5081 non-null   object 
 4   popularity        5081 non-null   int64  
 5   youtube_url       5081 non-null   object 
 6   tempo             5081 non-null   float64
 7   energy            5081 non-null   float64
 8   spectral_rolloff  5081 non-null   float64
 9   chroma            5081 non-null   float64
 10  danceability      5081 non-null   float64
dtypes: float64(5), int64(1), object(5)
memory usage: 436.8+ KB


In [3]:
# Dropping exact duplicates
data = data.drop_duplicates()  # Call the drop_duplicates function

# Drop duplicates with the same name while keeping the first occurrence of each song
data = data.sort_values(by=['popularity'], ascending=False)
data.drop_duplicates(subset=['name'], keep='first', inplace=True)

# Drop duplicates with the same song_id while keeping the first occurrence of each song
data = data.drop_duplicates(subset='song_id', keep='first')


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3310 entries, 2597 to 5080
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   song_id           3310 non-null   object 
 1   name              3310 non-null   object 
 2   artist            3310 non-null   object 
 3   album             3310 non-null   object 
 4   popularity        3310 non-null   int64  
 5   youtube_url       3310 non-null   object 
 6   tempo             3310 non-null   float64
 7   energy            3310 non-null   float64
 8   spectral_rolloff  3310 non-null   float64
 9   chroma            3310 non-null   float64
 10  danceability      3310 non-null   float64
dtypes: float64(5), int64(1), object(5)
memory usage: 310.3+ KB


In [5]:
# applying scaling to energy, chroma and danceability so the range can be between 0 to 1
from sklearn.preprocessing import MinMaxScaler

# Define the features to scale
features = ['energy', 'chroma', 'danceability']

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Fit and transform the selected features, storing them back in the data dataframe
data[features] = scaler.fit_transform(data[features])


In [6]:
data.head()

,song_id,name,artist,album,popularity,youtube_url,tempo,energy,spectral_rolloff,chroma,danceability
2597,2plbrEY59IikOBgBGLjaoe,Die With A Smile,Lady Gaga,Die With A Smile,100,https://www.youtube.com/watch?v=kPa7bsKwL-c,151.999081,0.275263,4553.724520,0.206741,0.685975
3015,6dOtVTDdiauQNBQEDOtlAB,BIRDS OF A FEATHER,Billie Eilish,HIT ME HARD AND SOFT,97,https://www.youtube.com/watch?v=d5gf9dXbPi0,103.359375,0.473026,4111.013014,0.377576,0.199982
2748,0WbMK4wrZ1wFSty9F7FCgu,"Good Luck, Babe!",Chappell Roan,"Good Luck, Babe!",92,https://www.youtube.com/watch?v=1RKqOmSkGgM,117.453835,0.436596,4749.354118,0.240505,0.203657
2781,5G2f63n7IPVPPjfNIGih7Q,Taste,Sabrina Carpenter,Short n' Sweet,90,https://www.youtube.com/watch?v=z9Q9OzL_wI8,112.347147,0.515997,6210.176734,0.330740,0.238436
2548,7tI8dRuH2Yc6RuoTjxo4dU,Who,Jimin,MUSE,90,https://www.youtube.com/watch?v=dk7NkMLYAis,117.453835,0.547213,7159.904111,0.291583,0.258594


In [7]:
# Saving the clean data into a csv file
data.to_csv('music_data.csv', index=False)